# Homework and bake-off: Relation extraction using distant supervision

In [2]:
__author__ = "Bill MacCartney and Christopher Potts"
__version__ = "CS224u, Stanford, Spring 2020"

## Contents

1. [Overview](#Overview)
1. [Set-up](#Set-up)
1. [Baselines](#Baselines)
  1. [Hand-build feature functions](#Hand-build-feature-functions)
  1. [Distributed representations](#Distributed-representations)
1. [Homework questions](#Homework-questions)
  1. [Different model factory [1 points]](#Different-model-factory-[1-points])
  1. [Directional unigram features [1.5 points]](#Directional-unigram-features-[1.5-points])
  1. [The part-of-speech tags of the "middle" words [1.5 points]](#The-part-of-speech-tags-of-the-"middle"-words-[1.5-points])
  1. [Bag of Synsets [2 points]](#Bag-of-Synsets-[2-points])
  1. [Your original system [3 points]](#Your-original-system-[3-points])
1. [Bake-off [1 point]](#Bake-off-[1-point])

## Overview

This homework and associated bake-off are devoted to developing really effective relation extraction systems using distant supervision. 

As with the previous assignments, this notebook first establishes a baseline system. The initial homework questions ask you to create additional baselines and suggest areas for innovation, and the final homework question asks you to develop an original system for you to enter into the bake-off.

## Set-up

See [the first notebook in this unit](rel_ext_01_task.ipynb#Set-up) for set-up instructions.

In [1]:
import numpy as np
import os
import rel_ext
from sklearn.linear_model import LogisticRegression
import utils

As usual, we unite our corpus and KB into a dataset, and create some splits for experimentation:

In [3]:
rel_ext_data_home = os.path.join('data', 'rel_ext_data')

In [4]:
corpus = rel_ext.Corpus(os.path.join(rel_ext_data_home, 'corpus.tsv.gz'))

In [5]:
kb = rel_ext.KB(os.path.join(rel_ext_data_home, 'kb.tsv.gz'))

In [6]:
dataset = rel_ext.Dataset(corpus, kb)

You are not wedded to this set-up for splits. The bake-off will be conducted on a previously unseen test-set, so all of the data in `dataset` is fair game:

In [7]:
splits = dataset.build_splits(
    split_names=['tiny', 'train', 'dev'],
    split_fracs=[0.01, 0.79, 0.20],
    seed=1)

In [8]:
splits

{'tiny': Corpus with 3,474 examples; KB with 445 triples,
 'train': Corpus with 263,285 examples; KB with 36,191 triples,
 'dev': Corpus with 64,937 examples; KB with 9,248 triples,
 'all': Corpus with 331,696 examples; KB with 45,884 triples}

## Baselines

### Hand-build feature functions

In [2]:
def simple_bag_of_words_featurizer(kbt, corpus, feature_counter):
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    return feature_counter

In [10]:
featurizers = [simple_bag_of_words_featurizer]

In [11]:
model_factory = lambda: LogisticRegression(fit_intercept=True, solver='liblinear')

In [12]:
baseline_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=featurizers,
    model_factory=model_factory,
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.898      0.388      0.711        340       5716
author                    0.845      0.527      0.754        509       5885
capital                   0.536      0.158      0.362         95       5471
contains                  0.789      0.602      0.743       3904       9280
film_performance          0.785      0.550      0.723        766       6142
founders                  0.842      0.405      0.692        380       5756
genre                     0.540      0.159      0.365        170       5546
has_sibling               0.826      0.246      0.562        499       5875
has_spouse                0.868      0.322      0.648        594       5970
is_a                      0.690      0.241      0.503        497       5873
nationality               0.638      0.199      0.443        301       5677
parents     

Studying model weights might yield insights:

In [13]:
rel_ext.examine_model_weights(baseline_results)

Highest and lowest feature weights for relation adjoins:

     2.558 Córdoba
     2.491 Taluks
     2.464 Valais
     ..... .....
    -1.362 based
    -1.512 America
    -2.308 Earth

Highest and lowest feature weights for relation author:

     2.484 author
     2.423 wrote
     2.192 by
     ..... .....
    -2.296 played
    -3.663 17th
    -3.674 1945

Highest and lowest feature weights for relation capital:

     2.593 capital
     1.850 posted
     1.552 especially
     ..... .....
    -1.464 borough
    -2.044 Province
    -2.076 Isfahan

Highest and lowest feature weights for relation contains:

     2.485 bordered
     2.112 attended
     1.954 southwestern
     ..... .....
    -2.471 Henley-on-Thames
    -2.548 who
    -6.027 Bronx

Highest and lowest feature weights for relation film_performance:

     4.131 starring
     3.802 co-starring
     3.263 opposite
     ..... .....
    -1.992 Westminster
    -2.155 Anjaani
    -2.155 Anjaana

Highest and lowest feature weights for 

### Distributed representations

This simple baseline sums the GloVe vector representations for all of the words in the "middle" span and feeds those representations into the standard `LogisticRegression`-based `model_factory`. The crucial parameter that enables this is `vectorize=False`. This essentially says to `rel_ext.experiment` that your featurizer or your model will do the work of turning examples into vectors; in that case, `rel_ext.experiment` just organizes these representations by relation type.

In [14]:
GLOVE_HOME = os.path.join('data', 'glove.6B')

In [15]:
glove_lookup = utils.glove2dict(
    os.path.join(GLOVE_HOME, 'glove.6B.300d.txt'))

In [9]:
def glove_middle_featurizer(kbt, corpus, np_func=np.sum):
    reps = []
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split():
            rep = glove_lookup.get(word)
            if rep is not None:
                reps.append(rep)
    # A random representation of the right dimensionality if the
    # example happens not to overlap with GloVe's vocabulary:
    if len(reps) == 0:
        dim = len(next(iter(glove_lookup.values())))                
        return utils.randvec(n=dim)
    else:
        return np_func(reps, axis=0)

In [17]:
glove_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=[glove_middle_featurizer],    
    vectorize=False, # Crucial for this featurizer!
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.873      0.444      0.732        340       5716
author                    0.852      0.442      0.719        509       5885
capital                   0.548      0.179      0.388         95       5471
contains                  0.656      0.414      0.587       3904       9280
film_performance          0.784      0.322      0.610        766       6142
founders                  0.744      0.245      0.528        380       5756
genre                     0.579      0.065      0.224        170       5546
has_sibling               0.792      0.244      0.547        499       5875
has_spouse                0.835      0.350      0.654        594       5970
is_a                      0.731      0.153      0.416        497       5873
nationality               0.692      0.209      0.474        301       5677
parents     

With the same basic code design, one can also use the PyTorch models included in the course repo, or write new ones that are better aligned with the task. For those models, it's likely that the featurizer will just return a list of tokens (or perhaps a list of lists of tokens), and the model will map those into vectors using an embedding.

## Homework questions

Please embed your homework responses in this notebook, and do not delete any cells from the notebook. (You are free to add as many cells as you like as part of your responses.)

### Different model factory [1 points]

The code in `rel_ext` makes it very easy to experiment with other classifier models: one need only redefine the `model_factory` argument. This question asks you to assess a [Support Vector Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html).

__To submit:__ A wrapper function `run_svm_model_factory` that does the following: 

1. Uses `rel_ext.experiment` with the model factory set to one based in an `SVC` with `kernel='linear'` and all other arguments left with default values. 
1. Trains on the 'train' part of `splits`.
1. Assesses on the `dev` part of `splits`.
1. Uses `featurizers` as defined above. 
1. Returns the return value of `rel_ext.experiment` for this set-up.

The function `test_run_svm_model_factory` will check that your function conforms to these general specifications.

In [18]:
def run_svm_model_factory():
    
    ##### YOUR CODE HERE
    from sklearn.svm import SVC
    model_factory = lambda: SVC(kernel='linear')
    
    featurizers = [simple_bag_of_words_featurizer]
    
    svc_results = rel_ext.experiment(
        splits,
        train_split='train',
        test_split='dev',
        featurizers=featurizers,
        model_factory=model_factory,
        verbose=True)
    
    return svc_results



In [19]:
def test_run_svm_model_factory(run_svm_model_factory):
    results = run_svm_model_factory()
    assert 'featurizers' in results, \
        "The return value of `run_svm_model_factory` seems not to be correct"
    # Check one of the models to make sure it's an SVC:
    assert 'SVC' in results['models']['adjoins'].__class__.__name__, \
        "It looks like the model factor wasn't set to use an SVC."    

In [20]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_run_svm_model_factory(run_svm_model_factory)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.850      0.332      0.648        340       5716
author                    0.769      0.589      0.725        509       5885
capital                   0.659      0.284      0.521         95       5471
contains                  0.771      0.612      0.733       3904       9280
film_performance          0.765      0.621      0.731        766       6142
founders                  0.791      0.429      0.677        380       5756
genre                     0.512      0.247      0.422        170       5546
has_sibling               0.779      0.226      0.524        499       5875
has_spouse                0.834      0.355      0.657        594       5970
is_a                      0.592      0.292      0.491        497       5873
nationality               0.542      0.193      0.398        301       5677
parents     

### Directional unigram features [1.5 points]

The current bag-of-words representation makes no distinction between "forward" and "reverse" examples. But, intuitively, there is big difference between _X and his son Y_ and _Y and his son X_. This question asks you to modify `simple_bag_of_words_featurizer` to capture these differences. 

__To submit:__

1. A feature function `directional_bag_of_words_featurizer` that is just like `simple_bag_of_words_featurizer` except that it distinguishes "forward" and "reverse". To do this, you just need to mark each word feature for whether it is derived from a subject–object example or from an object–subject example.  The included function `test_directional_bag_of_words_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `directional_bag_of_words_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment` as exemplified above in this notebook.)

3. `rel_ext.experiment` returns some of the core objects used in the experiment. How many feature names does the `vectorizer` have for the experiment run in the previous step? Include the code needed for getting this value. (Note: we're partly asking you to figure out how to get this value by using the sklearn documentation, so please don't ask how to do it!)

In [10]:
def directional_bag_of_words_featurizer(kbt, corpus, feature_counter): 
    # Append these to the end of the keys you add/access in 
    # `feature_counter` to distinguish the two orders. You'll
    # need to use exactly these strings in order to pass 
    # `test_directional_bag_of_words_featurizer`.
    subject_object_suffix = "_SO"
    object_subject_suffix = "_OS"
    
    ##### YOUR CODE HERE
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split(' '):
            feature_counter[word+subject_object_suffix] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            feature_counter[word+object_subject_suffix] += 1
    return feature_counter


# Call to `rel_ext.experiment`:
##### YOUR CODE HERE    

featurizers = [directional_bag_of_words_featurizer]
model_factory = lambda: LogisticRegression(fit_intercept=True, solver='liblinear')
baseline_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=featurizers,
    model_factory=model_factory,
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.843      0.441      0.713        340       5716
author                    0.820      0.591      0.761        509       5885
capital                   0.639      0.242      0.481         95       5471
contains                  0.817      0.648      0.777       3904       9280
film_performance          0.845      0.663      0.801        766       6142
founders                  0.799      0.408      0.670        380       5756
genre                     0.703      0.306      0.558        170       5546
has_sibling               0.855      0.248      0.575        499       5875
has_spouse                0.855      0.357      0.668        594       5970
is_a                      0.709      0.249      0.518        497       5873
nationality               0.696      0.236      0.501        301       5677
parents     

In [22]:
print("Number of feature names: ", len(baseline_results['vectorizer'].get_feature_names()))

Number of feature names:  40589


In [12]:
def test_directional_bag_of_words_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter['is_OS'] += 5
    feature_counter = directional_bag_of_words_featurizer(kbt, corpus, feature_counter)
    expected = defaultdict(
        int, {'is_OS':6,'a_OS':1,'webcomic_OS':1,'created_OS':1,'by_OS':1})
    assert feature_counter == expected, \
        "Expected:\n{}\nGot:\n{}".format(expected, feature_counter)

In [24]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_directional_bag_of_words_featurizer(corpus)

### The part-of-speech tags of the "middle" words [1.5 points]

Our corpus distribution contains part-of-speech (POS) tagged versions of the core text spans. Let's begin to explore whether there is information in these sequences, focusing on `middle_POS`.

__To submit:__

1. A feature function `middle_bigram_pos_tag_featurizer` that is just like `simple_bag_of_words_featurizer` except that it creates a feature for bigram POS sequences. For example, given 

  `The/DT dog/N napped/V`
  
   we obtain the list of bigram POS sequences
  
   `b = ['<s> DT', 'DT N', 'N V', 'V </s>']`. 
   
   Of course, `middle_bigram_pos_tag_featurizer` should return count dictionaries defined in terms of such bigram POS lists, on the model of `simple_bag_of_words_featurizer`.  Don't forget the start and end tags, to model those environments properly! The included function `test_middle_bigram_pos_tag_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `middle_bigram_pos_tag_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment` as exemplified above in this notebook.)

In [11]:
def middle_bigram_pos_tag_featurizer(kbt, corpus, feature_counter):
    
    ##### YOUR CODE HERE
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        tags = get_tags(ex.middle_POS)
        for word in get_tag_bigrams(tags):
            feature_counter[word] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        tags = get_tags(ex.middle_POS)
        for word in get_tag_bigrams(tags):
            feature_counter[word] += 1
    return feature_counter

def get_tag_bigrams(s):
    """Suggested helper method for `middle_bigram_pos_tag_featurizer`.
    This should be defined so that it returns a list of str, where each 
    element is a POS bigram."""
    # The values of `start_symbol` and `end_symbol` are defined
    # here so that you can use `test_middle_bigram_pos_tag_featurizer`.
    start_symbol = "<s>"
    end_symbol = "</s>"
    
    ##### YOUR CODE HERE
    s = [start_symbol] + s + [end_symbol]
    bigram_list = []
    for x, y in zip(s, s[1:]): 
        bigram_list.append(x+' '+y)
        
    return bigram_list


def get_tags(s): 
    """Given a sequence of word/POS elements (lemmas), this function
    returns a list containing just the POS elements, in order.    
    """
    return [parse_lem(lem)[1] for lem in s.strip().split(' ') if lem]


def parse_lem(lem):
    """Helper method for parsing word/POS elements. It just splits
    on the rightmost / and returns (word, POS) as a tuple of str."""
    return lem.strip().rsplit('/', 1)  

# Call to `rel_ext.experiment`:
##### YOUR CODE HERE
featurizers = [middle_bigram_pos_tag_featurizer]
model_factory = lambda: LogisticRegression(fit_intercept=True, solver='liblinear')
baseline_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=featurizers,
    model_factory=model_factory,
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.815      0.350      0.644        340       5716
author                    0.704      0.346      0.583        509       5885
capital                   0.516      0.168      0.365         95       5471
contains                  0.753      0.589      0.713       3904       9280
film_performance          0.706      0.441      0.630        766       6142
founders                  0.550      0.187      0.396        380       5756
genre                     0.682      0.176      0.434        170       5546
has_sibling               0.638      0.180      0.423        499       5875
has_spouse                0.761      0.273      0.560        594       5970
is_a                      0.518      0.171      0.369        497       5873
nationality               0.373      0.073      0.205        301       5677
parents     

In [26]:
def test_middle_bigram_pos_tag_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter['<s> VBZ'] += 5
    feature_counter = middle_bigram_pos_tag_featurizer(kbt, corpus, feature_counter)
    expected = defaultdict(
        int, {'<s> VBZ':6,'VBZ DT':1,'DT JJ':1,'JJ VBN':1,'VBN IN':1,'IN </s>':1})
    assert feature_counter == expected, \
        "Expected:\n{}\nGot:\n{}".format(expected, feature_counter)

In [27]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_middle_bigram_pos_tag_featurizer(corpus)

### Bag of Synsets [2 points]

The following allows you to use NLTK's WordNet API to get the synsets compatible with _dog_ as used as a noun:

```
from nltk.corpus import wordnet as wn
dog = wn.synsets('dog', pos='n')
dog
[Synset('dog.n.01'),
 Synset('frump.n.01'),
 Synset('dog.n.03'),
 Synset('cad.n.01'),
 Synset('frank.n.02'),
 Synset('pawl.n.01'),
 Synset('andiron.n.01')]
```

This question asks you to create synset-based features from the word/tag pairs in `middle_POS`.

__To submit:__

1. A feature function `synset_featurizer` that is just like `simple_bag_of_words_featurizer` except that it returns a list of synsets derived from `middle_POS`. Stringify these objects with `str` so that they can be `dict` keys. Use `convert_tag` (included below) to convert tags to `pos` arguments usable by `wn.synsets`. The included function `test_synset_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `synset_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment`.)

In [12]:
from nltk.corpus import wordnet as wn

def synset_featurizer(kbt, corpus, feature_counter):
    
    ##### YOUR CODE HERE
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in get_synsets(ex.middle_POS):
            feature_counter[word] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in get_synsets(ex.middle_POS):
            feature_counter[word] += 1
    return feature_counter


def get_synsets(s):
    """Suggested helper method for `synset_featurizer`. This should
    be completed so that it returns a list of stringified Synsets 
    associated with elements of `s`.
    """   
    # Use `parse_lem` from the previous question to get a list of
    # (word, POS) pairs. Remember to convert the POS strings.
    wt = [parse_lem(lem) for lem in s.strip().split(' ') if lem]
    
    ##### YOUR CODE HERE
    
    from nltk.corpus import wordnet as wn

    w_synsets = []
    for i in range(len(wt)):
        current_synsets = [str(x) for x in wn.synsets(wt[i][0], pos=convert_tag(wt[i][1]))]
        w_synsets += current_synsets
        
    w_synsets = set(w_synsets)
    
    return w_synsets
    
    
def convert_tag(t):
    """Converts tags so that they can be used by WordNet:
    
    | Tag begins with | WordNet tag |
    |-----------------|-------------|
    | `N`             | `n`         |
    | `V`             | `v`         |
    | `J`             | `a`         |
    | `R`             | `r`         |
    | Otherwise       | `None`      |
    """        
    if t[0].lower() in {'n', 'v', 'r'}:
        return t[0].lower()
    elif t[0].lower() == 'j':
        return 'a'
    else:
        return None    


# Call to `rel_ext.experiment`:
##### YOUR CODE HERE    

featurizers = [synset_featurizer]
model_factory = lambda: LogisticRegression(fit_intercept=True, solver='liblinear')
baseline_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=featurizers,
    model_factory=model_factory,
    verbose=True)


/home/ps/anaconda3/envs/nlu/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.785      0.332      0.617        340       5716
author                    0.783      0.462      0.688        509       5885
capital                   0.533      0.168      0.372         95       5471
contains                  0.772      0.578      0.724       3904       9280
film_performance          0.796      0.546      0.729        766       6142
founders                  0.755      0.374      0.627        380       5756
genre                     0.412      0.206      0.343        170       5546
has_sibling               0.786      0.220      0.519        499       5875
has_spouse                0.844      0.310      0.628        594       5970
is_a                      0.585      0.227      0.445        497       5873
nationality               0.517      0.150      0.347        301       5677
parents     

In [29]:
def test_synset_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter["Synset('be.v.01')"] += 5
    feature_counter = synset_featurizer(kbt, corpus, feature_counter)
    # The full return values for this tend to be long, so we just
    # test a few examples to avoid cluttering up this notebook.
    test_cases = {
        "Synset('be.v.01')": 6,
        "Synset('embody.v.02')": 1
    }
    for ss, expected in test_cases.items():   
        result = feature_counter[ss]
        assert result == expected, \
            "Incorrect count for {}: Expected {}; Got {}".format(ss, expected, result)

In [1]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_synset_featurizer(corpus)

NameError: name 'os' is not defined

### Your original system [3 points]

There are many options, and this could easily grow into a project. Here are a few ideas:

- Try out different classifier models, from `sklearn` and elsewhere.
- Add a feature that indicates the length of the middle.
- Augment the bag-of-words representation to include bigrams or trigrams (not just unigrams).
- Introduce features based on the entity mentions themselves. <!-- \[SPOILER: it helps a lot, maybe 4% in F-score. And combines nicely with the directional features.\] -->
- Experiment with features based on the context outside (rather than between) the two entity mentions — that is, the words before the first mention, or after the second.
- Try adding features which capture syntactic information, such as the dependency-path features used by Mintz et al. 2009. The [NLTK](https://www.nltk.org/) toolkit contains a variety of [parsing algorithms](http://www.nltk.org/api/nltk.parse.html) that may help.
- The bag-of-words representation does not permit generalization across word categories such as names of people, places, or companies. Can we do better using word embeddings such as [GloVe](https://nlp.stanford.edu/projects/glove/)?

In the cell below, please provide a brief technical description of your original system, so that the teaching team can gain an understanding of what it does. This will help us to understand your code and analyze all the submissions to identify patterns and strategies.

In [13]:
# Enter your system description in this cell.
# This is my code description attached here as requested
# We implement six additional featurizers, namely length of the middle words and bigrams/trigrams of the 
# bag-of-words representation, and left and right window size of 1 and 5 featurizers including POS information. 
# After that, we combine all the featurizers we have implemented in above and feed into the experiment function 
# to do training. Adding left and right context improves the overall system significantly, interestingly note, 
# featurizing using 5-gram doesn't outperform unigram of size of 5. In addition, including window size of 1 and 5 
# outperforms having only unigram window size of 5. This could due to the system give more weight to the immediate left and right 
# word of the two entity mentions.

# My peak score was: 0.669
# This is my code
def length_words_featurizer(kbt, corpus, feature_counter):
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        feature_counter[str(len(ex.middle))] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        feature_counter[str(len(ex.middle))] += 1
    return feature_counter
    

def complex_bag_of_words_featurizer(kbt, corpus, feature_counter):
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        unigram_words = ex.middle.split(' ')
        bigram_words = get_ngram(unigram_words, n=2)
        trigram_words = get_ngram(unigram_words, n=3)
        
        for word in unigram_words:
            feature_counter[word] += 1
        for word in bigram_words:
            feature_counter[word] += 1
        for word in trigram_words:
            feature_counter[word] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        unigram_words = ex.middle.split(' ')
        bigram_words = get_ngram(unigram_words, n=2)
        trigram_words = get_ngram(unigram_words, n=3)
        
        for word in unigram_words:
            feature_counter[word] += 1
        for word in bigram_words:
            feature_counter[word] += 1
        for word in trigram_words:
            feature_counter[word] += 1
    return feature_counter


def get_ngram(s, n):
    start_symbol = "<s>"
    end_symbol = "</s>"
    
    s = [start_symbol] + s + [end_symbol]
    ngram_list = []
    if n == 2:
        for x, y in zip(s, s[1:]): 
            ngram_list.append(x+' '+y)
    elif n == 3:
        if len(s) < 3:
            return ["<s> </s>"]
        for x, y, z in zip(s, s[1:], s[2:]): 
            ngram_list.append(x+' '+y+' '+z)
        
    return ngram_list


def left_window_size_5_featurizer(kbt, corpus, feature_counter):
    window_size = 5
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        word_pos_pair = ex.left_POS.split(' ')
        if len(word_pos_pair):
            for i in range(window_size):
                try:
                    w_pos = word_pos_pair[-i].split('/')
                    if len(w_pos) == 2:
                        w = w_pos[0]
                        p = w_pos[1]
                        feature_counter[w + ' ' + p] += 1 # word
                except:
                    break

        
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        word_pos_pair = ex.left_POS.split(' ')
        if len(word_pos_pair):
            for i in range(window_size):
                try:
                    w_pos = word_pos_pair[-i].split('/')
                    if len(w_pos) == 2:
                        w = w_pos[0]
                        p = w_pos[1]
                        feature_counter[w + ' ' + p] += 1 # word
                except:
                    break
    return feature_counter

def right_window_size_5_featurizer(kbt, corpus, feature_counter):
    window_size = 5
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        word_pos_pair = ex.right_POS.split(' ')
        if len(word_pos_pair):
            for i in range(window_size):
                try:
                    w_pos = word_pos_pair[i].split('/')
                    if len(w_pos) == 2:
                        w = w_pos[0]
                        p = w_pos[1]
                        feature_counter[w + ' ' + p] += 1 # word
                except:
                    break

        
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        word_pos_pair = ex.right_POS.split(' ')
        if len(word_pos_pair):
            for i in range(window_size):
                try:
                    w_pos = word_pos_pair[i].split('/')
                    if len(w_pos) == 2:
                        w = w_pos[0]
                        p = w_pos[1]
                        feature_counter[w + ' ' + p] += 1 # word
                except:
                    break

    return feature_counter


def left_window_size_1_featurizer(kbt, corpus, feature_counter):
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        word_pos_pair = ex.left_POS.split(' ')
        if len(word_pos_pair):
            w_pos = word_pos_pair[-1].split('/')
            if len(w_pos) == 2:
                w = w_pos[0]
                p = w_pos[1]
                feature_counter[w + ' ' + p] += 1 # word
        
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        word_pos_pair = ex.left_POS.split(' ')
        if len(word_pos_pair):
            w_pos = word_pos_pair[-1].split('/')
            if len(w_pos) == 2:
                w = w_pos[0]
                p = w_pos[1]
                feature_counter[w + ' ' + p] += 1 # word
    return feature_counter

def right_window_size_1_featurizer(kbt, corpus, feature_counter):
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        word_pos_pair = ex.right_POS.split(' ')
        if len(word_pos_pair):
            w_pos = word_pos_pair[0].split('/')
            if len(w_pos) == 2:
                w = w_pos[0]
                p = w_pos[1]
                feature_counter[w + ' ' + p] += 1 # word
        
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        word_pos_pair = ex.right_POS.split(' ')
        if len(word_pos_pair):
            w_pos = word_pos_pair[0].split('/')
            if len(w_pos) == 2:
                w = w_pos[0]
                p = w_pos[1]
                feature_counter[w + ' ' + p] += 1 # word
    return feature_counter


if 'IS_GRADESCOPE_ENV' not in os.environ:
    featurizers = [left_window_size_5_featurizer, right_window_size_5_featurizer, left_window_size_1_featurizer, right_window_size_1_featurizer,complex_bag_of_words_featurizer, length_words_featurizer, directional_bag_of_words_featurizer,middle_bigram_pos_tag_featurizer,synset_featurizer]
    model_factory = lambda: LogisticRegression(fit_intercept=True, solver='liblinear')

    baseline_results = rel_ext.experiment(
        splits,
        train_split='train',
        test_split='dev',
        featurizers=featurizers,
        model_factory=model_factory,
        verbose=True)

# Please do not remove this comment.

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.849      0.447      0.720        340       5716
author                    0.829      0.676      0.793        509       5885
capital                   0.550      0.232      0.431         95       5471
contains                  0.845      0.837      0.843       3904       9280
film_performance          0.847      0.731      0.821        766       6142
founders                  0.764      0.434      0.663        380       5756
genre                     0.738      0.465      0.661        170       5546
has_sibling               0.859      0.391      0.693        499       5875
has_spouse                0.794      0.396      0.661        594       5970
is_a                      0.642      0.382      0.565        497       5873
nationality               0.725      0.684      0.717        301       5677
parents     

## Bake-off [1 point]

For the bake-off, we will release a test set. The announcement will go out on the discussion forum. You will evaluate your custom model from the previous question on these new datasets using the function `rel_ext.bake_off_experiment`. Rules:

1. Only one evaluation is permitted.
1. No additional system tuning is permitted once the bake-off has started.

The cells below this one constitute your bake-off entry.

People who enter will receive the additional homework point, and people whose systems achieve the top score will receive an additional 0.5 points. We will test the top-performing systems ourselves, and only systems for which we can reproduce the reported results will win the extra 0.5 points.

Late entries will be accepted, but they cannot earn the extra 0.5 points. Similarly, you cannot win the bake-off unless your homework is submitted on time.

The announcement will include the details on where to submit your entry.

In [18]:
# Enter your bake-off assessment code in this cell. 
# Please do not remove this comment.
if 'IS_GRADESCOPE_ENV' not in os.environ:
    # Please enter your code in the scope of the above conditional.
    ##### YOUR CODE HERE
    rel_ext_data_home_test = os.path.join(
        rel_ext_data_home, 'bakeoff-rel_ext-test-data')

    featurizers = [left_window_size_5_featurizer, right_window_size_5_featurizer, left_window_size_1_featurizer, right_window_size_1_featurizer,complex_bag_of_words_featurizer, length_words_featurizer, directional_bag_of_words_featurizer,middle_bigram_pos_tag_featurizer,synset_featurizer]
    model_factory = lambda: LogisticRegression(fit_intercept=True, solver='liblinear')

    bakeoff_results = rel_ext.experiment(
        splits,
        train_split='train',
        test_split='dev',
        featurizers=featurizers,
        model_factory=model_factory,
        verbose=False)
    
    rel_ext.bake_off_experiment(bakeoff_results, rel_ext_data_home_test)


relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.879      0.516      0.771        438       7122
author                    0.850      0.678      0.809        645       7329
capital                   0.647      0.383      0.568        115       6799
contains                  0.830      0.797      0.823       3808      10492
film_performance          0.876      0.777      0.855       1011       7695
founders                  0.773      0.482      0.689        444       7128
genre                     0.773      0.399      0.651        188       6872
has_sibling               0.878      0.350      0.674        717       7401
has_spouse                0.858      0.432      0.716        780       7464
is_a                      0.707      0.391      0.609        611       7295
nationality               0.542      0.319      0.475        383       7067
parents     

In [33]:
# On an otherwise blank line in this cell, please enter
# your macro-average f-score (an F_0.5 score) as reported 
# by the code above. Please enter only a number between 
# 0 and 1 inclusive. Please do not remove this comment.
if 'IS_GRADESCOPE_ENV' not in os.environ:
    pass
    # Please enter your score in the scope of the above conditional.
    ##### YOUR CODE HERE
    0.666


